# Load the needed libraries

In [1]:
import pandas as pd
import numpy as np
from time import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.utils import resample
from sklearn import metrics
from sklearn.metrics import adjusted_rand_score
import os
import sys

/Users/ra1077ba/miniconda3/envs/tf2_env/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Deep Learning libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Reshape, Input, Concatenate, Embedding
from tensorflow.keras.callbacks import ModelCheckpoint

#from tensorflow.keras.engine.topology import Layer, InputSpec
from tensorflow.keras.layers import Layer, InputSpec

from tensorflow.keras import initializers, optimizers
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras import callbacks

# Additional TensorFlow-specific imports if needed
# Example:
# from tensorflow.keras.layers import ...

# Custom Layer or Model classes need to be updated to inherit from tensorflow.keras classes instead of keras
# Example:
# class CustomLayer(tf.keras.layers.Layer):
#     ...


In [3]:
# Seeding libraries
from numpy.random import seed
# from tensorflow import set_random_seed
from tensorflow.keras.utils import set_random_seed
import random
# import seaborn as sns; sns.set()

In [4]:
# validation libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [5]:
p = os.path.abspath('/Users/ra1077ba/Documents/DataScience/GU/AsthmaClusteringMuwada/Python_Functions')
if p not in sys.path:
    sys.path.append(p)

In [6]:
# importing functions form our project
from DEC import cluster_acc, autoencoder, ClusteringLayer, DEC
from FcDEC import FcDEC
from FcIDEC import FcIDEC
from IDEC import IDEC

# Validation of IDEC method
1. We will first split the data to modeling and validation sets
2. We will cluster the modeling data and add the cluster labels to the modeling data. (Here we will learn a IDEC cluster model)
3. We will split the modeling data to training and testing
4. We will train a RF model on the training data and test on the test data
5. We will cluster the validation set (By using the model developed in 2) and then uses the model developed in 4 to predict the labels
6. compare the results of 5

In [7]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn import metrics
# we can as develop this further by allowing for dec function for instance
def get_val_res(Processed_data, clus_n, loss_fun):
    # split the data as in step 1
    model_IDEC, valid_IDEC = train_test_split(Processed_data, test_size=0.2)
    # step 2
    optimizer = Adam()
    idec_allerg_val = FcIDEC(dims= [model_IDEC.shape[-1], 200, 10], n_clusters=clus_n)
    idec_allerg_val.pretrain(model_IDEC, batch_size=128, epochs=200, optimizer=optimizer, aug_pretrain=False)
    idec_allerg_val.compile(optimizer=optimizer, loss= loss_fun)
    y_model = idec_allerg_val.fit(model_IDEC.values, y=None, maxiter=10000, batch_size=128, update_interval=40, aug_cluster=False)
    # step 3
    X_train_IDEC, X_test_IDEC, y_train_IDEC, y_test_IDEC = train_test_split(model_IDEC, y_model, test_size=0.2) # 80%
    # step 4
    # Create a Gaussian Classifier
    clf_IDEC=RandomForestClassifier(n_estimators=1000)
    # Train the model using the training sets y_pred=clf.predict(X_test)
    clf_IDEC.fit(X_train_IDEC,y_train_IDEC)
    predValid_IDEC=clf_IDEC.predict(X_test_IDEC)
    # step 5
    # cluster the validation data before using the random forest classifier
    y_pred_val_IDEC = idec_allerg_val.predict_labels(valid_IDEC)
    # step 6
    # measure the accuraccy between the clustering on the test data and the prediction from the random forest model
    predValid_val_IDEC=clf_IDEC.predict(valid_IDEC)
    return metrics.accuracy_score(y_pred_val_IDEC, predValid_val_IDEC)
    

# One hot encoded data 3 clusters

In [6]:
# pd.set_option('display.max_columns', 30)
OH_data = pd.read_csv('../Intermediates/SavedObjects/OH_data.csv')
del OH_data['Unnamed: 0']
OH_data.head()

,gina_steps_onlys saba or non users,gina_steps_step 1 or 2,gina_steps_step3,gina_steps_step4 or 5,gina_control_controlled,gina_control_partially controlled,gina_control_uncontrolled,education_primary,education_secondary,education_tertiary,...,smoking_exposure_at_home_Yes,smoking_exposure_at_work_Yes,household_damage_Yes,exertion_triggered_asthma_Yes,infection_triggered_asthma_Yes,allergen_triggered_asthma_Yes,irritant_triggered_asthma_Yes,weather_triggered_asthma_Yes,emotion_triggered_asthma_Yes,XCOPD_Yes
0,0.564225,-0.389721,-0.198859,-0.277642,-0.874468,1.108573,-0.365756,1.038973,-0.806310,-0.378112,...,-1.272229,1.224877,2.720553,1.256725,1.280964,2.122861,-0.318080,-0.495785,-0.237025,-0.311274
1,-1.771772,2.565109,-0.198859,-0.277642,-0.874468,1.108573,-0.365756,1.038973,-0.806310,-0.378112,...,0.785769,-0.816146,-0.367454,1.256725,-0.780411,-0.470910,-0.318080,-0.495785,-0.237025,-0.311274
2,0.564225,-0.389721,-0.198859,-0.277642,1.143184,-0.901769,-0.365756,1.038973,-0.806310,-0.378112,...,0.785769,1.224877,-0.367454,1.256725,1.280964,2.122861,3.142851,2.016354,-0.237025,-0.311274
3,0.564225,-0.389721,-0.198859,-0.277642,-0.874468,1.108573,-0.365756,-0.962179,1.239817,-0.378112,...,0.785769,-0.816146,-0.367454,-0.795463,-0.780411,-0.470910,-0.318080,-0.495785,-0.237025,-0.311274
4,-1.771772,-0.389721,5.027078,-0.277642,-0.874468,1.108573,-0.365756,-0.962179,1.239817,-0.378112,...,-1.272229,-0.816146,2.720553,-0.795463,-0.780411,-0.470910,-0.318080,-0.495785,-0.237025,-0.311274


## IDEC

In [33]:
# have reproducible behavior for certain hash-based operations.
os.environ['PYTHONHASHSEED'] = str(0)
# The below is necessary for starting Numpy generated random numbers in a well-defined initial state.
random.seed(55)
# The below is necessary for starting core Python generated random numbers in a well-defined state.
seed(33)
# Force TensorFlow to use single thread. Multiple threads are a potential source of non-reproducible results.
# session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
set_random_seed(458)
# sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
# K.set_session(sess)

In [106]:
set_random_seed(458)
# dims=[OH_data.shape[-1], 1000, 10] where 1000 is number of neourons and 10 is the dimension of the latent(embeddings) spacce. It is also possible 
# to use multi layers for example dims=[OH_data.shape[-1], 1000,100, 10] where 100 is the number of nuorons in the layers 2 of the model. 
optimizer = Adam()
idec_OH = FcIDEC(dims= [OH_data.shape[-1], 1000, 10], n_clusters=3)
# pretrain the modelloss=['kld', 'mse']loss=['kld', 'mse']
idec_OH.pretrain(OH_data, batch_size=128, epochs=500, optimizer=optimizer, aug_pretrain=False)
# compile
idec_OH.compile(optimizer=optimizer, loss='kld')
y_pred_OH = idec_OH.fit(OH_data.values, y=None, maxiter=10000, batch_size=128, update_interval=40, aug_cluster=False)

/Users/ra1077ba/miniconda3/envs/tf2_env/lib/python3.8/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Begin pretraining:  ------------------------------------------------------------
Epoch 1/500
25/25 [==============================] - 0s 2ms/step - loss: 0.7665
Epoch 2/500
25/25 [==============================] - 0s 1ms/step - loss: 0.5711
Epoch 3/500
25/25 [==============================] - 0s 2ms/step - loss: 0.5159
Epoch 4/500
25/25 [==============================] - 0s 2ms/step - loss: 0.4838
Epoch 5/500
25/25 [==============================] - 0s 5ms/step - loss: 0.4647
Epoch 6/500
25/25 [==============================] - 0s 2ms/step - loss: 0.4516
Epoch 7/500
25/25 [==============================] - 0s 2ms/step - loss: 0.4418
Epoch 8/500
25/25 [==============================] - 0s 2ms/step - loss: 0.4329
Epoch 9/500
25/25 [==============================] - 0s 2ms/step - loss: 0.4260
Epoch 10/500
25/25 [==============================] - 0s 2ms/step - loss: 0.4174
Epoch 11/500
25/25 [==============================] - 0s 2ms/step - loss: 0.4117
Epoch 12/500
25/25 [=================

Pretraining time:  23.75185203552246
Pretrained weights are saved to results/temp/ae_weights.h5
End pretraining:  ------------------------------------------------------------
Begin clustering: ------------------------------------------------------------
Update interval 40
Save interval 10000
Initializing cluster centers with k-means.
97/97 [==============================] - 0s 314us/step
saving model to: ./results/temp/model_0.h5
delta_label  0.0009674298613350532 < tol  0.001
Reached tolerance threshold. Stopping training.
saving model to: ./results/temp/model_final.h5
Clustering time: 8s
End clustering: ------------------------------------------------------------


In [107]:
np.unique(y_pred_OH, return_counts= True)

(array([0, 1, 2]), array([ 527, 1524, 1050]))

In [97]:
np.unique(y_pred_OH, return_counts= True)

(array([0, 1, 2]), array([ 712, 1633,  756]))

In [99]:
np.unique(y_pred_OH, return_counts= True)

(array([0, 1, 2]), array([ 381, 1630, 1090]))

In [58]:
np.unique(y_pred_OH, return_counts= True)

(array([0, 1, 2]), array([ 528, 1079, 1494]))

In [86]:
[OH_data.shape[-1], 200, 10][0], OH_data.shape

(53, (3101, 53))

In [87]:
idec_OH.encoder(OH_data.values).shape, idec_OH.encoder.predict(OH_data.values).shape

97/97 [==============================] - 0s 331us/step


(TensorShape([3101, 10]), (3101, 10))

## DEC

In [59]:
dec_OH = DEC(dims=[OH_data.shape[-1], 1000, 10], n_clusters=3)
#plot_model(dec_food.model, to_file='dec_model.png', show_shapes=True)
dec_OH.model.summary()

t0 = time()

# Pretrain autoencoders before clustering
dec_OH.pretrain(OH_data, batch_size=128, epochs=40, optimizer='adam')
# begin clustering, time not include pretraining part.

dec_OH.compile(loss='kld', optimizer='adam')
dec_OH.fit(OH_data, y=None, batch_size=128, tol= 0.00001, maxiter=15000,update_interval=40)

# Show the final results
y_pred_dec_OH = dec_OH.y_pred
#print('acc:', cluster_acc(y, y_pred))
print('clustering time: %d seconds.' % int(time() - t0))

Model: "model_84"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 53)]              0         
                                                                 
 encoder_0 (Dense)           (None, 1000)              54000     
                                                                 
 encoder_1 (Dense)           (None, 10)                10010     
                                                                 
 clustering (ClusteringLaye  (None, 3)                 30        
 r)                                                              
                                                                 
Total params: 64040 (250.16 KB)
Trainable params: 64040 (250.16 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
...Pretraining...
Epoch 1/40
25/25 [==============================] - 0s 1ms/step - lo

In [48]:
np.unique(y_pred_dec_OH, return_counts= True)

(array([0, 1, 2]), array([1025,  723, 1353]))

In [60]:
np.unique(y_pred_dec_OH, return_counts= True)

(array([0, 1, 2]), array([1023,  718, 1360]))

In [19]:
vals_mse_IDEC_OH = [get_val_res(Processed_data=OH_data, clus_n=3, loss_fun='mse') for _ in range(5)]
vals_kld_IDEC = [get_val_res(Processed_data=OH_data, clus_n=3, loss_fun='kld') for _ in range(5)]

/Users/ra1077ba/miniconda3/envs/tf2_env/lib/python3.8/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Begin pretraining:  ------------------------------------------------------------
Epoch 1/200
20/20 [==============================] - 0s 935us/step - loss: 0.9355
Epoch 2/200
20/20 [==============================] - 0s 735us/step - loss: 0.7867
Epoch 3/200
20/20 [==============================] - 0s 663us/step - loss: 0.6704
Epoch 4/200
20/20 [==============================] - 0s 735us/step - loss: 0.6126
Epoch 5/200
20/20 [==============================] - 0s 782us/step - loss: 0.5808
Epoch 6/200
20/20 [==============================] - 0s 859us/step - loss: 0.5599
Epoch 7/200
20/20 [==============================] - 0s 2ms/step - loss: 0.5428
Epoch 8/200
20/20 [==============================] - 0s 1ms/step - loss: 0.5275
Epoch 9/200
20/20 [==============================] - 0s 888us/step - loss: 0.5149
Epoch 10/200
20/20 [==============================] - 0s 824us/step - loss: 0.5025
Epoch 11/200
20/20 [==============================] - 0s 850us/step - loss: 0.4928
Epoch 12/200
20/20 

 3.969151020050049
Pretrained weights are saved to results/temp/ae_weights.h5
End pretraining:  ------------------------------------------------------------
Begin clustering: ------------------------------------------------------------
Update interval 40
Save interval 10000
Initializing cluster centers with k-means.
78/78 [==============================] - 0s 245us/step
saving model to: ./results/temp/model_0.h5
delta_label  0.0008064516129032258 < tol  0.001
Reached tolerance threshold. Stopping training.
saving model to: ./results/temp/model_final.h5
Clustering time: 2s
End clustering: ------------------------------------------------------------


/Users/ra1077ba/miniconda3/envs/tf2_env/lib/python3.8/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Begin pretraining:  ------------------------------------------------------------
Epoch 1/200
20/20 [==============================] - 0s 712us/step - loss: 0.9339
Epoch 2/200
20/20 [==============================] - 0s 730us/step - loss: 0.7751
Epoch 3/200
20/20 [==============================] - 0s 693us/step - loss: 0.6747
Epoch 4/200
20/20 [==============================] - 0s 693us/step - loss: 0.6096
Epoch 5/200
20/20 [==============================] - 0s 763us/step - loss: 0.5736
Epoch 6/200
20/20 [==============================] - 0s 844us/step - loss: 0.5516
Epoch 7/200
20/20 [==============================] - 0s 831us/step - loss: 0.5339
Epoch 8/200
20/20 [==============================] - 0s 846us/step - loss: 0.5187
Epoch 9/200
20/20 [==============================] - 0s 845us/step - loss: 0.5053
Epoch 10/200
20/20 [==============================] - 0s 825us/step - loss: 0.4938
Epoch 11/200
20/20 [==============================] - 0s 853us/step - loss: 0.4846
Epoch 12/200
20

...

Begin pretraining:  ------------------------------------------------------------
Epoch 1/200
20/20 [==============================] - 0s 747us/step - loss: 0.9414
Epoch 2/200
20/20 [==============================] - 0s 711us/step - loss: 0.7834
Epoch 3/200
20/20 [==============================] - 0s 717us/step - loss: 0.6633
Epoch 4/200
20/20 [==============================] - 0s 698us/step - loss: 0.6044
Epoch 5/200
20/20 [==============================] - 0s 14ms/step - loss: 0.5739
Epoch 6/200
20/20 [==============================] - 0s 905us/step - loss: 0.5512
Epoch 7/200
20/20 [==============================] - 0s 866us/step - loss: 0.5337
Epoch 8/200
20/20 [==============================] - 0s 863us/step - loss: 0.5180
Epoch 9/200
20/20 [==============================] - 0s 810us/step - loss: 0.5038
Epoch 10/200
20/20 [==============================] - 0s 898us/step - loss: 0.4915
Epoch 11/200
20/20 [==============================] - 0s 873us/step - loss: 0.4801
Epoch 12/200
20/

Pretraining time:  4.286349058151245
Pretrained weights are saved to results/temp/ae_weights.h5
End pretraining:  ------------------------------------------------------------
Begin clustering: ------------------------------------------------------------
Update interval 40
Save interval 10000
Initializing cluster centers with k-means.
78/78 [==============================] - 0s 348us/step
saving model to: ./results/temp/model_0.h5
delta_label  0.0004032258064516129 < tol  0.001
Reached tolerance threshold. Stopping training.
saving model to: ./results/temp/model_final.h5
Clustering time: 2s
End clustering: ------------------------------------------------------------


/Users/ra1077ba/miniconda3/envs/tf2_env/lib/python3.8/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Begin pretraining:  ------------------------------------------------------------
Epoch 1/200
20/20 [==============================] - 0s 784us/step - loss: 0.9361
Epoch 2/200
20/20 [==============================] - 0s 760us/step - loss: 0.7613
Epoch 3/200
20/20 [==============================] - 0s 716us/step - loss: 0.6657
Epoch 4/200
20/20 [==============================] - 0s 823us/step - loss: 0.6084
Epoch 5/200
20/20 [==============================] - 0s 885us/step - loss: 0.5753
Epoch 6/200
20/20 [==============================] - 0s 950us/step - loss: 0.5542
Epoch 7/200
20/20 [==============================] - 0s 904us/step - loss: 0.5363
Epoch 8/200
20/20 [==============================] - 0s 2ms/step - loss: 0.5215
Epoch 9/200
20/20 [==============================] - 0s 943us/step - loss: 0.5091
Epoch 10/200
20/20 [==============================] - 0s 937us/step - loss: 0.4975
Epoch 11/200
20/20 [==============================] - 0s 893us/step - loss: 0.4869
Epoch 12/200
20/2

Pretraining time:  4.0851099491119385
Pretrained weights are saved to results/temp/ae_weights.h5
End pretraining:  ------------------------------------------------------------
Begin clustering: ------------------------------------------------------------
Update interval 40
Save interval 10000
Initializing cluster centers with k-means.
78/78 [==============================] - 0s 276us/step
saving model to: ./results/temp/model_0.h5
delta_label  0.0008064516129032258 < tol  0.001
Reached tolerance threshold. Stopping training.
saving model to: ./results/temp/model_final.h5
Clustering time: 0s
End clustering: ------------------------------------------------------------


In [20]:
vals_mse_IDEC_OH, vals_kld_IDEC

([0.9484702093397746,
  0.9210950080515298,
  0.9420289855072463,
  0.9243156199677939,
  0.9355877616747182],
 [0.9887278582930756,
  0.9468599033816425,
  0.9243156199677939,
  0.9259259259259259,
  0.9565217391304348])

## Saving results

In [97]:
result_OH_IDEC_3 = OH_data.copy()
result_OH_IDEC_3['cluster'] = y_pred_OH
result_OH_IDEC_3 = result_OH_IDEC_3.to_csv('../Intermediates/SavedObjects/result_OH_IDEC_3.csv', index = None, header=True)

In [ ]:
result_OH_DEC_3 = OH_data.copy()
result_OH_DEC_3['cluster'] = y_pred_dec_OH
result_OH_DEC_3 = result_OH_DEC_3.to_csv('../Intermediates/SavedObjects/result_OH_DEC_3.csv', index = None, header=True)

# One hot encoded data 4 clusters

## IDEC

In [61]:
optimizer = Adam()
idec_OH_4 = FcIDEC(dims= [OH_data.shape[-1], 1000, 10], n_clusters=4)
# pretrain the modelloss=['kld', 'mse']loss=['kld', 'mse']
idec_OH_4.pretrain(OH_data, batch_size=128, epochs=200, optimizer=optimizer, aug_pretrain=False)
# compile
idec_OH_4.compile(optimizer=optimizer, loss='kld')
y_pred_OH_4 = idec_OH_4.fit(OH_data.values, y=None, maxiter=10000, batch_size=128, update_interval=40, aug_cluster=False)

/Users/ra1077ba/miniconda3/envs/tf2_env/lib/python3.8/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Begin pretraining:  ------------------------------------------------------------
Epoch 1/200
25/25 [==============================] - 0s 2ms/step - loss: 0.7666
Epoch 2/200
25/25 [==============================] - 0s 2ms/step - loss: 0.5816
Epoch 3/200
25/25 [==============================] - 0s 2ms/step - loss: 0.5287
Epoch 4/200
25/25 [==============================] - 0s 2ms/step - loss: 0.4941
Epoch 5/200
25/25 [==============================] - 0s 2ms/step - loss: 0.4721
Epoch 6/200
25/25 [==============================] - 0s 2ms/step - loss: 0.4574
Epoch 7/200
25/25 [==============================] - 0s 2ms/step - loss: 0.4461
Epoch 8/200
25/25 [==============================] - 0s 2ms/step - loss: 0.4370
Epoch 9/200
25/25 [==============================] - 0s 2ms/step - loss: 0.4288
Epoch 10/200
25/25 [==============================] - 0s 2ms/step - loss: 0.4217
Epoch 11/200
25/25 [==============================] - 0s 2ms/step - loss: 0.4152
Epoch 12/200
25/25 [=================

Pretraining time:  10.006940841674805
Pretrained weights are saved to results/temp/ae_weights.h5
End pretraining:  ------------------------------------------------------------
Begin clustering: ------------------------------------------------------------
Update interval 40
Save interval 10000
Initializing cluster centers with k-means.
97/97 [==============================] - 0s 327us/step
saving model to: ./results/temp/model_0.h5
delta_label  0.0003224766204450177 < tol  0.001
Reached tolerance threshold. Stopping training.
saving model to: ./results/temp/model_final.h5
Clustering time: 8s
End clustering: ------------------------------------------------------------


In [120]:
idec_OH_4.predict_labels(OH_data)

array([2, 2, 2, ..., 3, 1, 3])

In [62]:
np.unique(y_pred_OH_4, return_counts= True)

(array([0, 1, 2, 3]), array([ 341,  417,  907, 1436]))

In [23]:
vals_mse_IDEC_OH_4 = [get_val_res(Processed_data=OH_data, clus_n=4, loss_fun='mse') for _ in range(5)]
vals_kld_IDEC_4 = [get_val_res(Processed_data=OH_data, clus_n=4, loss_fun='kld') for _ in range(5)]

/Users/ra1077ba/miniconda3/envs/tf2_env/lib/python3.8/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Begin pretraining:  ------------------------------------------------------------
Epoch 1/200
20/20 [==============================] - 0s 959us/step - loss: 0.9419
Epoch 2/200
20/20 [==============================] - 0s 776us/step - loss: 0.7601
Epoch 3/200
20/20 [==============================] - 0s 761us/step - loss: 0.6598
Epoch 4/200
20/20 [==============================] - 0s 728us/step - loss: 0.6062
Epoch 5/200
20/20 [==============================] - 0s 781us/step - loss: 0.5766
Epoch 6/200
20/20 [==============================] - 0s 893us/step - loss: 0.5533
Epoch 7/200
20/20 [==============================] - 0s 879us/step - loss: 0.5347
Epoch 8/200
20/20 [==============================] - 0s 838us/step - loss: 0.5179
Epoch 9/200
20/20 [==============================] - 0s 887us/step - loss: 0.5030
Epoch 10/200
20/20 [==============================] - 0s 859us/step - loss: 0.4896
Epoch 11/200
20/20 [==============================] - 0s 872us/step - loss: 0.4782
Epoch 12/200
20

Pretraining time:  4.177788734436035
Pretrained weights are saved to results/temp/ae_weights.h5
End pretraining:  ------------------------------------------------------------
Begin clustering: ------------------------------------------------------------
Update interval 40
Save interval 10000
Initializing cluster centers with k-means.
78/78 [==============================] - 0s 275us/step
saving model to: ./results/temp/model_0.h5
delta_label  0.0008064516129032258 < tol  0.001
Reached tolerance threshold. Stopping training.
saving model to: ./results/temp/model_final.h5
Clustering time: 1s
End clustering: ------------------------------------------------------------


/Users/ra1077ba/miniconda3/envs/tf2_env/lib/python3.8/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Begin pretraining:  ------------------------------------------------------------
Epoch 1/200
20/20 [==============================] - 0s 911us/step - loss: 0.9275
Epoch 2/200
20/20 [==============================] - 0s 858us/step - loss: 0.7626
Epoch 3/200
20/20 [==============================] - 0s 810us/step - loss: 0.6623
Epoch 4/200
20/20 [==============================] - 0s 889us/step - loss: 0.6017
Epoch 5/200
20/20 [==============================] - 0s 897us/step - loss: 0.5688
Epoch 6/200
20/20 [==============================] - 0s 969us/step - loss: 0.5479
Epoch 7/200
20/20 [==============================] - 0s 947us/step - loss: 0.5313
Epoch 8/200
20/20 [==============================] - 0s 953us/step - loss: 0.5172
Epoch 9/200
20/20 [==============================] - 0s 1ms/step - loss: 0.5047
Epoch 10/200
20/20 [==============================] - 0s 964us/step - loss: 0.4940
Epoch 11/200
20/20 [==============================] - 0s 883us/step - loss: 0.4850
Epoch 12/200
20/2

...

Begin pretraining:  ------------------------------------------------------------
Epoch 1/200
20/20 [==============================] - 0s 976us/step - loss: 0.9312
Epoch 2/200
20/20 [==============================] - 0s 910us/step - loss: 0.7669
Epoch 3/200
20/20 [==============================] - 0s 766us/step - loss: 0.6725
Epoch 4/200
20/20 [==============================] - 0s 833us/step - loss: 0.6119
Epoch 5/200
20/20 [==============================] - 0s 953us/step - loss: 0.5722
Epoch 6/200
20/20 [==============================] - 0s 958us/step - loss: 0.5471
Epoch 7/200
20/20 [==============================] - 0s 922us/step - loss: 0.5279
Epoch 8/200
20/20 [==============================] - 0s 2ms/step - loss: 0.5108
Epoch 9/200
20/20 [==============================] - 0s 1ms/step - loss: 0.4955
Epoch 10/200
20/20 [==============================] - 0s 1ms/step - loss: 0.4829
Epoch 11/200
20/20 [==============================] - 0s 922us/step - loss: 0.4719
Epoch 12/200
20/20 [=

Pretrained weights are saved to results/temp/ae_weights.h5
End pretraining:  ------------------------------------------------------------
Begin clustering: ------------------------------------------------------------
Update interval 40
Save interval 10000
Initializing cluster centers with k-means.
78/78 [==============================] - 0s 296us/step
saving model to: ./results/temp/model_0.h5
delta_label  0.0004032258064516129 < tol  0.001
Reached tolerance threshold. Stopping training.
saving model to: ./results/temp/model_final.h5
Clustering time: 1s
End clustering: ------------------------------------------------------------


/Users/ra1077ba/miniconda3/envs/tf2_env/lib/python3.8/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Begin pretraining:  ------------------------------------------------------------
Epoch 1/200
20/20 [==============================] - 0s 1ms/step - loss: 0.9441
Epoch 2/200
20/20 [==============================] - 0s 922us/step - loss: 0.7641
Epoch 3/200
20/20 [==============================] - 0s 814us/step - loss: 0.6700
Epoch 4/200
20/20 [==============================] - 0s 811us/step - loss: 0.6145
Epoch 5/200
20/20 [==============================] - 0s 833us/step - loss: 0.5793
Epoch 6/200
20/20 [==============================] - 0s 893us/step - loss: 0.5556
Epoch 7/200
20/20 [==============================] - 0s 983us/step - loss: 0.5369
Epoch 8/200
20/20 [==============================] - 0s 884us/step - loss: 0.5222
Epoch 9/200
20/20 [==============================] - 0s 863us/step - loss: 0.5089
Epoch 10/200
20/20 [==============================] - 0s 888us/step - loss: 0.4978
Epoch 11/200
20/20 [==============================] - 0s 856us/step - loss: 0.4876
Epoch 12/200
20/2

Pretraining time:  4.098057985305786
Pretrained weights are saved to results/temp/ae_weights.h5
End pretraining:  ------------------------------------------------------------
Begin clustering: ------------------------------------------------------------
Update interval 40
Save interval 10000
Initializing cluster centers with k-means.
78/78 [==============================] - 0s 290us/step
saving model to: ./results/temp/model_0.h5
delta_label  0.0008064516129032258 < tol  0.001
Reached tolerance threshold. Stopping training.
saving model to: ./results/temp/model_final.h5
Clustering time: 1s
End clustering: ------------------------------------------------------------


In [24]:
vals_mse_IDEC_OH_4, vals_kld_IDEC_4

([0.9307568438003221,
  0.9388083735909822,
  0.9500805152979066,
  0.9066022544283414,
  0.9629629629629629],
 [0.9355877616747182,
  0.9581320450885669,
  0.9098228663446055,
  0.9436392914653784,
  0.9227053140096618])

## DEC 

In [63]:
dec_OH_4 = DEC(dims=[OH_data.shape[-1], 1000, 10], n_clusters=4)
#plot_model(dec_food.model, to_file='dec_model.png', show_shapes=True)
dec_OH.model.summary()

t0 = time()

# Pretrain autoencoders before clustering
dec_OH_4.pretrain(OH_data, batch_size=128, epochs=40, optimizer='adam')
# begin clustering, time not include pretraining part.

dec_OH_4.compile(loss='kld', optimizer='adam')
dec_OH_4.fit(OH_data, y=None, batch_size=128, tol= 0.00001, maxiter=15000,update_interval=40)

# Show the final results
y_pred_dec_OH_4 = dec_OH_4.y_pred
#print('acc:', cluster_acc(y, y_pred))
print('clustering time: %d seconds.' % int(time() - t0))

Model: "model_84"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 53)]              0         
                                                                 
 encoder_0 (Dense)           (None, 1000)              54000     
                                                                 
 encoder_1 (Dense)           (None, 10)                10010     
                                                                 
 clustering (ClusteringLaye  (None, 3)                 30        
 r)                                                              
                                                                 
Total params: 64040 (250.16 KB)
Trainable params: 64040 (250.16 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
...Pretraining...
Epoch 1/40
25/25 [==============================] - 0s 1ms/step - lo

In [64]:
np.unique(y_pred_dec_OH_4, return_counts= True)

(array([0, 1, 2, 3]), array([1339,  459,  609,  694]))

## Saving the results

In [ ]:
result_OH_IDEC_4 = OH_data.copy()
result_OH_IDEC_4['cluster'] = y_pred_OH_4
result_OH_IDEC_4 = result_OH_IDEC_3.to_csv('../Intermediates/SavedObjects/result_OH_IDEC_4.csv', index = None, header=True)

In [ ]:
result_OH_DEC_4 = OH_data.copy()
result_OH_DEC_4['cluster'] = y_pred_dec_OH_4
result_OH_DEC_4 = result_OH_IDEC_3.to_csv('../Intermediates/SavedObjects/result_OH_DEC_4.csv', index = None, header=True)

# One hot encodeed data without aux variables 3 cluster

## Custom clustring function

In [8]:
def cluster_data(data, n_clusters, neurons=[1000, 10], model_type='FcIDEC', epochs=200, batch_size=128, maxiter=10000, update_interval=40, tol=0.00001):
    """
    Apply clustering using either FcIDEC or DEC to the provided data.

    :param data: The data to be clustered.
    :param n_clusters: The number of clusters to use.
    :param neurons: List of neuron counts for the network layers.
    :param model_type: The type of model to use ('FcIDEC' or 'DEC').
    :param epochs: The number of epochs for pretraining.
    :param batch_size: The batch size for training.
    :param maxiter: The maximum number of iterations for clustering.
    :param update_interval: The interval for updating clusters.
    :param tol: Tolerance for stopping criterion.
    :return: The predicted cluster labels.
    """
    set_random_seed(458)
    optimizer = Adam()

    # Construct the dims list based on input neurons
    dims = [data.shape[-1]] + neurons

    # Initialize the model
    if model_type == 'FcIDEC':
        model = FcIDEC(dims=dims, n_clusters=n_clusters)
        epochs = 200  # FcIDEC specific default
    elif model_type == 'DEC':
        model = DEC(dims=dims, n_clusters=n_clusters)
        epochs = 40   # DEC specific default
    else:
        raise ValueError("Invalid model type. Choose 'FcIDEC' or 'DEC'.")

    # Pretrain the model
    model.pretrain(data, batch_size=batch_size, epochs=epochs, optimizer=optimizer)

    # Compile the model
    # mse or kld
    model.compile(optimizer=optimizer, loss='mse')

    # Fit the model
    if model_type == 'FcIDEC':
        model.fit(data.values, y=None, maxiter=maxiter, batch_size=batch_size, update_interval=update_interval, aug_cluster=False)
        y_pred = model.predict_labels(data.values)
    elif model_type == 'DEC':
        model.fit(data, y=None, batch_size=batch_size, tol=tol, maxiter=maxiter, update_interval=update_interval)
        y_pred = model.predict_clusters(data)

    return y_pred


In [191]:
# pd.set_option('display.max_columns', 30)
OH_data_No_aux = pd.read_csv('../Intermediates/SavedObjects/OH_data_no_aux.csv')
del OH_data_No_aux['Unnamed: 0']
OH_data_No_aux.head()

,gina_steps,education,smoking_status,Age,BMI,onset_age_asthma,FEV1_FVC_ratio_post,Reversibility,tlco,kco,...,smoking_exposure_at_home_Yes,smoking_exposure_at_work_Yes,household_damage_Yes,exertion_triggered_asthma_Yes,infection_triggered_asthma_Yes,allergen_triggered_asthma_Yes,irritant_triggered_asthma_Yes,weather_triggered_asthma_Yes,emotion_triggered_asthma_Yes,XCOPD_Yes
0,1,1,1,30,21.728395,7,0.780767,4.429347,120.376880,105.486292,...,0,1,1,1,1,1,0,0,0,0
1,2,1,1,28,22.595641,3,0.853547,6.240592,121.049749,118.599749,...,1,0,0,1,0,0,0,0,0,0
2,1,1,2,35,27.120316,13,0.864331,12.741017,123.995834,112.723202,...,1,1,0,1,1,1,1,1,0,0
3,1,2,3,31,23.624447,-1,0.880236,1.150785,108.383562,101.488270,...,1,0,0,0,0,0,0,0,0,0
4,3,2,1,42,29.377583,-1,0.764283,1.546211,95.803697,103.289899,...,0,0,1,0,0,0,0,0,0,0


In [193]:
# OH_data_No_aux.dtypes
# Load the numerical variable names
num_var = pd.read_csv('../Intermediates/SavedObjects/numerical_vars.csv')
# Convert the names to a list
num_var_list = num_var.iloc[:, 0].tolist()

In [192]:
# Assuming num_var_list is already defined and contains the names of numerical columns

# Scale only the numerical columns
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_numerical_data = scaler.fit_transform(OH_data_No_aux[num_var_list])

# Create a DataFrame from the scaled numerical data
scaled_numerical_df = pd.DataFrame(scaled_numerical_data, columns=num_var_list)

# Drop the original numerical columns from the main DataFrame
OH_data_No_aux = OH_data_No_aux.drop(columns=num_var_list)

# Concatenate the scaled numerical DataFrame with the main DataFrame
OH_data_No_aux_scaled = pd.concat([OH_data_No_aux, scaled_numerical_df], axis=1)
OH_data_No_aux_scaled

,gina_steps,education,smoking_status,Gender_Yes,Cough_Yes,Breathlessness_Yes,Recurrent_wheeze_Yes,dyspenea_Yes,Chronic_productive_cough_Yes,spt_any_Yes,...,Reversibility,tlco,kco,FeNO,Eosinophiles,Neutrophiles,Lymphocytes,Monocytes,Basophiles,pack_years
0,1,1,1,0,0,0,0,0,0,1,...,-0.226771,1.018486,0.540072,0.090893,-0.674407,1.885105,0.520085,-0.785421,-0.626121,-0.394858
1,2,1,1,1,0,1,0,0,0,1,...,0.043672,1.051811,1.413522,-0.327198,1.340369,-1.377155,-0.250277,0.528061,1.626055,-0.843092
2,1,1,2,1,0,1,0,1,0,1,...,1.014272,1.197722,1.022102,-0.546696,-0.674407,-0.561590,-1.020640,-0.128680,-0.626121,-0.245446
3,1,2,3,1,1,1,0,0,1,1,...,-0.716304,0.424494,0.273775,-0.682575,-0.808726,-0.858159,-1.328785,-0.785421,-0.626121,-0.824416
4,3,2,1,0,0,1,0,0,1,1,...,-0.657262,-0.198548,0.393777,0.529888,-0.674407,-0.561590,-0.558422,-0.128680,-0.626121,-0.768387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3096,4,1,2,0,1,0,0,0,0,0,...,-1.054962,-1.102941,-1.623112,-0.123379,-0.002815,0.698829,-0.558422,1.841543,-0.626121,3.415138
3097,1,1,1,0,0,0,0,0,0,0,...,-0.412864,1.505967,0.811434,-0.055439,-0.741567,-1.191800,-1.174712,-1.442161,-0.626121,-0.745975
3098,1,1,1,0,0,0,0,0,0,0,...,-0.220613,0.213484,-0.054992,0.258129,-0.808726,-0.932302,-0.866567,-1.442161,-0.626121,-0.320152
3099,1,2,3,0,0,0,0,0,0,0,...,-0.824049,1.396669,2.614273,-0.316746,0.668777,-0.042594,0.319791,0.528061,1.626055,-0.006387


In [165]:
# scaler = StandardScaler()
# scaled_array = scaler.fit_transform(OH_data_No_aux)
# OH_data_No_aux_scalled = pd.DataFrame(scaled_array, columns=OH_data_No_aux.columns)

In [166]:
# OH_data.values, OH_data_No_aux.values
type(OH_data), type(OH_data_No_aux), type(OH_data_No_aux_scalled)

(pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame)

In [71]:
vals_mse_IDEC_OH_No_aux_3 = [get_val_res(Processed_data=OH_data_No_aux, clus_n=3, loss_fun='mse') for _ in range(5)]
vals_kld_IDEC_No_aux_3 = [get_val_res(Processed_data=OH_data_No_aux, clus_n=3, loss_fun='kld') for _ in range(5)]

/Users/ra1077ba/miniconda3/envs/tf2_env/lib/python3.8/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Begin pretraining:  ------------------------------------------------------------
Epoch 1/200
20/20 [==============================] - 0s 867us/step - loss: 0.9203
Epoch 2/200
20/20 [==============================] - 0s 726us/step - loss: 0.7449
Epoch 3/200
20/20 [==============================] - 0s 727us/step - loss: 0.6380
Epoch 4/200
20/20 [==============================] - 0s 753us/step - loss: 0.5643
Epoch 5/200
20/20 [==============================] - 0s 837us/step - loss: 0.5260
Epoch 6/200
20/20 [==============================] - 0s 931us/step - loss: 0.5010
Epoch 7/200
20/20 [==============================] - 0s 848us/step - loss: 0.4814
Epoch 8/200
20/20 [==============================] - 0s 825us/step - loss: 0.4638
Epoch 9/200
20/20 [==============================] - 0s 858us/step - loss: 0.4494
Epoch 10/200
20/20 [==============================] - 0s 991us/step - loss: 0.4376
Epoch 11/200
20/20 [==============================] - 0s 864us/step - loss: 0.4278
Epoch 12/200
20

Pretrained weights are saved to results/temp/ae_weights.h5
End pretraining:  ------------------------------------------------------------
Begin clustering: ------------------------------------------------------------
Update interval 40
Save interval 10000
Initializing cluster centers with k-means.
78/78 [==============================] - 0s 293us/step
saving model to: ./results/temp/model_0.h5
delta_label  0.0008064516129032258 < tol  0.001
Reached tolerance threshold. Stopping training.
saving model to: ./results/temp/model_final.h5
Clustering time: 2s
End clustering: ------------------------------------------------------------


/Users/ra1077ba/miniconda3/envs/tf2_env/lib/python3.8/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Begin pretraining:  ------------------------------------------------------------
Epoch 1/200
20/20 [==============================] - 0s 944us/step - loss: 0.9387
Epoch 2/200
20/20 [==============================] - 0s 804us/step - loss: 0.7464
Epoch 3/200
20/20 [==============================] - 0s 735us/step - loss: 0.6185
Epoch 4/200
20/20 [==============================] - 0s 760us/step - loss: 0.5541
Epoch 5/200
20/20 [==============================] - 0s 840us/step - loss: 0.5197
Epoch 6/200
20/20 [==============================] - 0s 924us/step - loss: 0.4958
Epoch 7/200
20/20 [==============================] - 0s 938us/step - loss: 0.4760
Epoch 8/200
20/20 [==============================] - 0s 900us/step - loss: 0.4597
Epoch 9/200
20/20 [==============================] - 0s 913us/step - loss: 0.4459
Epoch 10/200
20/20 [==============================] - 0s 939us/step - loss: 0.4352
Epoch 11/200
20/20 [==============================] - 0s 928us/step - loss: 0.4258
Epoch 12/200
20

...

Begin pretraining:  ------------------------------------------------------------
Epoch 1/200
20/20 [==============================] - 0s 826us/step - loss: 0.9493
Epoch 2/200
20/20 [==============================] - 0s 739us/step - loss: 0.7480
Epoch 3/200
20/20 [==============================] - 0s 725us/step - loss: 0.6375
Epoch 4/200
20/20 [==============================] - 0s 730us/step - loss: 0.5700
Epoch 5/200
20/20 [==============================] - 0s 778us/step - loss: 0.5348
Epoch 6/200
20/20 [==============================] - 0s 886us/step - loss: 0.5115
Epoch 7/200
20/20 [==============================] - 0s 904us/step - loss: 0.4919
Epoch 8/200
20/20 [==============================] - 0s 910us/step - loss: 0.4751
Epoch 9/200
20/20 [==============================] - 0s 881us/step - loss: 0.4598
Epoch 10/200
20/20 [==============================] - 0s 859us/step - loss: 0.4474
Epoch 11/200
20/20 [==============================] - 0s 906us/step - loss: 0.4372
Epoch 12/200
20

Pretrained weights are saved to results/temp/ae_weights.h5
End pretraining:  ------------------------------------------------------------
Begin clustering: ------------------------------------------------------------
Update interval 40
Save interval 10000
Initializing cluster centers with k-means.
78/78 [==============================] - 0s 272us/step
saving model to: ./results/temp/model_0.h5
delta_label  0.0008064516129032258 < tol  0.001
Reached tolerance threshold. Stopping training.
saving model to: ./results/temp/model_final.h5
Clustering time: 0s
End clustering: ------------------------------------------------------------


/Users/ra1077ba/miniconda3/envs/tf2_env/lib/python3.8/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Begin pretraining:  ------------------------------------------------------------
Epoch 1/200
20/20 [==============================] - 0s 930us/step - loss: 0.9338
Epoch 2/200
20/20 [==============================] - 0s 871us/step - loss: 0.7491
Epoch 3/200
20/20 [==============================] - 0s 998us/step - loss: 0.6428
Epoch 4/200
20/20 [==============================] - 0s 930us/step - loss: 0.5718
Epoch 5/200
20/20 [==============================] - 0s 872us/step - loss: 0.5329
Epoch 6/200
20/20 [==============================] - 0s 1ms/step - loss: 0.5073
Epoch 7/200
20/20 [==============================] - 0s 969us/step - loss: 0.4880
Epoch 8/200
20/20 [==============================] - 0s 921us/step - loss: 0.4706
Epoch 9/200
20/20 [==============================] - 0s 945us/step - loss: 0.4554
Epoch 10/200
20/20 [==============================] - 0s 990us/step - loss: 0.4428
Epoch 11/200
20/20 [==============================] - 0s 981us/step - loss: 0.4317
Epoch 12/200
20/2

Pretraining time:  4.237533092498779
Pretrained weights are saved to results/temp/ae_weights.h5
End pretraining:  ------------------------------------------------------------
Begin clustering: ------------------------------------------------------------
Update interval 40
Save interval 10000
Initializing cluster centers with k-means.
78/78 [==============================] - 0s 253us/step
saving model to: ./results/temp/model_0.h5
delta_label  0.0008064516129032258 < tol  0.001
Reached tolerance threshold. Stopping training.
saving model to: ./results/temp/model_final.h5
Clustering time: 1s
End clustering: ------------------------------------------------------------


In [72]:
vals_mse_IDEC_OH_No_aux_3, vals_kld_IDEC_No_aux_3

([0.9581320450885669,
  0.9694041867954911,
  0.9533011272141707,
  0.966183574879227,
  0.9694041867954911],
 [0.964573268921095,
  0.9371980676328503,
  0.9597423510466989,
  0.9468599033816425,
  0.9565217391304348])

In [67]:
vals_mse_IDEC_OH_No_aux_4 = [get_val_res(Processed_data=OH_data_No_aux, clus_n=4, loss_fun='mse') for _ in range(5)]
vals_kld_IDEC_No_aux_4 = [get_val_res(Processed_data=OH_data_No_aux, clus_n=4, loss_fun='kld') for _ in range(5)]

/Users/ra1077ba/miniconda3/envs/tf2_env/lib/python3.8/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Begin pretraining:  ------------------------------------------------------------
Epoch 1/200
20/20 [==============================] - 0s 912us/step - loss: 0.9293
Epoch 2/200
20/20 [==============================] - 0s 692us/step - loss: 0.7433
Epoch 3/200
20/20 [==============================] - 0s 717us/step - loss: 0.6415
Epoch 4/200
20/20 [==============================] - 0s 690us/step - loss: 0.5770
Epoch 5/200
20/20 [==============================] - 0s 804us/step - loss: 0.5389
Epoch 6/200
20/20 [==============================] - 0s 839us/step - loss: 0.5128
Epoch 7/200
20/20 [==============================] - 0s 831us/step - loss: 0.4920
Epoch 8/200
20/20 [==============================] - 0s 871us/step - loss: 0.4753
Epoch 9/200
20/20 [==============================] - 0s 887us/step - loss: 0.4609
Epoch 10/200
20/20 [==============================] - 0s 904us/step - loss: 0.4497
Epoch 11/200
20/20 [==============================] - 0s 969us/step - loss: 0.4402
Epoch 12/200
20

Pretrained weights are saved to results/temp/ae_weights.h5
End pretraining:  ------------------------------------------------------------
Begin clustering: ------------------------------------------------------------
Update interval 40
Save interval 10000
Initializing cluster centers with k-means.
78/78 [==============================] - 0s 290us/step
saving model to: ./results/temp/model_0.h5
delta_label  0.0008064516129032258 < tol  0.001
Reached tolerance threshold. Stopping training.
saving model to: ./results/temp/model_final.h5
Clustering time: 2s
End clustering: ------------------------------------------------------------


/Users/ra1077ba/miniconda3/envs/tf2_env/lib/python3.8/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Begin pretraining:  ------------------------------------------------------------
Epoch 1/200
20/20 [==============================] - 0s 786us/step - loss: 0.9256
Epoch 2/200
20/20 [==============================] - 0s 777us/step - loss: 0.7302
Epoch 3/200
20/20 [==============================] - 0s 748us/step - loss: 0.6245
Epoch 4/200
20/20 [==============================] - 0s 869us/step - loss: 0.5613
Epoch 5/200
20/20 [==============================] - 0s 777us/step - loss: 0.5285
Epoch 6/200
20/20 [==============================] - 0s 897us/step - loss: 0.5067
Epoch 7/200
20/20 [==============================] - 0s 920us/step - loss: 0.4885
Epoch 8/200
20/20 [==============================] - 0s 898us/step - loss: 0.4714
Epoch 9/200
20/20 [==============================] - 0s 896us/step - loss: 0.4561
Epoch 10/200
20/20 [==============================] - 0s 906us/step - loss: 0.4438
Epoch 11/200
20/20 [==============================] - 0s 846us/step - loss: 0.4329
Epoch 12/200
20

...

Begin pretraining:  ------------------------------------------------------------
Epoch 1/200
20/20 [==============================] - 0s 801us/step - loss: 0.9128
Epoch 2/200
20/20 [==============================] - 0s 747us/step - loss: 0.7288
Epoch 3/200
20/20 [==============================] - 0s 722us/step - loss: 0.6207
Epoch 4/200
20/20 [==============================] - 0s 719us/step - loss: 0.5614
Epoch 5/200
20/20 [==============================] - 0s 784us/step - loss: 0.5256
Epoch 6/200
20/20 [==============================] - 0s 901us/step - loss: 0.5029
Epoch 7/200
20/20 [==============================] - 0s 913us/step - loss: 0.4851
Epoch 8/200
20/20 [==============================] - 0s 901us/step - loss: 0.4687
Epoch 9/200
20/20 [==============================] - 0s 878us/step - loss: 0.4550
Epoch 10/200
20/20 [==============================] - 0s 993us/step - loss: 0.4433
Epoch 11/200
20/20 [==============================] - 0s 958us/step - loss: 0.4338
Epoch 12/200
20

Pretraining time:  4.232655048370361
Pretrained weights are saved to results/temp/ae_weights.h5
End pretraining:  ------------------------------------------------------------
Begin clustering: ------------------------------------------------------------
Update interval 40
Save interval 10000
Initializing cluster centers with k-means.
78/78 [==============================] - 0s 264us/step
saving model to: ./results/temp/model_0.h5
delta_label  0.0004032258064516129 < tol  0.001
Reached tolerance threshold. Stopping training.
saving model to: ./results/temp/model_final.h5
Clustering time: 1s
End clustering: ------------------------------------------------------------


/Users/ra1077ba/miniconda3/envs/tf2_env/lib/python3.8/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Begin pretraining:  ------------------------------------------------------------
Epoch 1/200
20/20 [==============================] - 0s 755us/step - loss: 0.9265
Epoch 2/200
20/20 [==============================] - 0s 705us/step - loss: 0.7444
Epoch 3/200
20/20 [==============================] - 0s 712us/step - loss: 0.6308
Epoch 4/200
20/20 [==============================] - 0s 686us/step - loss: 0.5691
Epoch 5/200
20/20 [==============================] - 0s 771us/step - loss: 0.5314
Epoch 6/200
20/20 [==============================] - 0s 853us/step - loss: 0.5058
Epoch 7/200
20/20 [==============================] - 0s 872us/step - loss: 0.4847
Epoch 8/200
20/20 [==============================] - 0s 844us/step - loss: 0.4669
Epoch 9/200
20/20 [==============================] - 0s 849us/step - loss: 0.4520
Epoch 10/200
20/20 [==============================] - 0s 825us/step - loss: 0.4403
Epoch 11/200
20/20 [==============================] - 0s 830us/step - loss: 0.4306
Epoch 12/200
20

Pretraining time:  3.9744298458099365
Pretrained weights are saved to results/temp/ae_weights.h5
End pretraining:  ------------------------------------------------------------
Begin clustering: ------------------------------------------------------------
Update interval 40
Save interval 10000
Initializing cluster centers with k-means.
78/78 [==============================] - 0s 279us/step
saving model to: ./results/temp/model_0.h5
delta_label  0.0008064516129032258 < tol  0.001
Reached tolerance threshold. Stopping training.
saving model to: ./results/temp/model_final.h5
Clustering time: 1s
End clustering: ------------------------------------------------------------


In [68]:
vals_mse_IDEC_OH_4, vals_kld_IDEC_4

([0.9581320450885669,
  0.9371980676328503,
  0.9468599033816425,
  0.9484702093397746,
  0.9533011272141707],
 [0.9452495974235104,
  0.9452495974235104,
  0.9146537842190016,
  0.9484702093397746,
  0.92914653784219])

In [200]:
# Define your dataset
OH_data_No_aux = OH_data_No_aux_scaled  # Load your dataset here

# Define the configurations
n_clusters_list = [3, 4, 6]
model_types = ['FcIDEC', 'DEC']
epochs = 1000

# Dictionary to store the results
results = {}

# Loop over each combination of n_clusters and model_type
for n_clusters in n_clusters_list:
    for model_type in model_types:
        # Key for storing results
        key = f"{model_type}_clusters_{n_clusters}"

        # Run the clustering function
        y_pred = cluster_data(data=OH_data_No_aux, n_clusters=n_clusters, neurons=[500, 20], model_type=model_type, epochs=epochs)

        # Store the results
        results[key] = y_pred

# Now, results dictionary contains all the outcomes
# Access the results like results['FcIDEC_clusters_3'], results['DEC_clusters_4'], etc.

/Users/ra1077ba/miniconda3/envs/tf2_env/lib/python3.8/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Begin pretraining:  ------------------------------------------------------------
Epoch 1/200
25/25 [==============================] - 0s 1ms/step - loss: 0.4721
Epoch 2/200
25/25 [==============================] - 0s 931us/step - loss: 0.2484
Epoch 3/200
25/25 [==============================] - 0s 1ms/step - loss: 0.1546
Epoch 4/200
25/25 [==============================] - 0s 1ms/step - loss: 0.1043
Epoch 5/200
25/25 [==============================] - 0s 2ms/step - loss: 0.0805
Epoch 6/200
25/25 [==============================] - 0s 1ms/step - loss: 0.0681
Epoch 7/200
25/25 [==============================] - 0s 1ms/step - loss: 0.0628
Epoch 8/200
25/25 [==============================] - 0s 1ms/step - loss: 0.0598
Epoch 9/200
25/25 [==============================] - 0s 1ms/step - loss: 0.0577
Epoch 10/200
25/25 [==============================] - 0s 1ms/step - loss: 0.0559
Epoch 11/200
25/25 [==============================] - 0s 1ms/step - loss: 0.0546
Epoch 12/200
25/25 [===============

Pretrained weights are saved to results/temp/ae_weights.h5
End pretraining:  ------------------------------------------------------------
Begin clustering: ------------------------------------------------------------
Update interval 40
Save interval 10000
Initializing cluster centers with k-means.
97/97 [==============================] - 0s 281us/step
saving model to: ./results/temp/model_0.h5


saving model to: ./results/temp/model_final.h5
Clustering time: 46s
End clustering: ------------------------------------------------------------
...Pretraining...
Epoch 1/40
25/25 [==============================] - 0s 1ms/step - loss: 0.4765
Epoch 2/40
25/25 [==============================] - 0s 938us/step - loss: 0.2466
Epoch 3/40
25/25 [==============================] - 0s 982us/step - loss: 0.1524
Epoch 4/40
25/25 [==============================] - 0s 1ms/step - loss: 0.1031
Epoch 5/40
25/25 [==============================] - 0s 3ms/step - loss: 0.0819
Epoch 6/40
25/25 [==============================] - 0s 2ms/step - loss: 0.0702
Epoch 7/40
25/25 [==============================] - 0s 1ms/step - loss: 0.0646
Epoch 8/40
25/25 [==============================] - 0s 1ms/step - loss: 0.0612
Epoch 9/40
25/25 [==============================] - 0s 1ms/step - loss: 0.0590
Epoch 10/40
25/25 [==============================] - 0s 1ms/step - loss: 0.0575
Epoch 11/40
25/25 [=======================

Pretrained weights are saved to ./ae_weights.h5
Update interval 40
Save interval 120
Initializing cluster centers with k-means.
97/97 [==============================] - 0s 291us/step
saving model to: ./results/dec/DEC_model_0.h5
saving model to: ./results/dec/DEC_model_120.h5
saving model to: ./results/dec/DEC_model_240.h5
saving model to: ./results/dec/DEC_model_360.h5
saving model to: ./results/dec/DEC_model_480.h5
saving model to: ./results/dec/DEC_model_600.h5
saving model to: ./results/dec/DEC_model_720.h5
saving model to: ./results/dec/DEC_model_840.h5
saving model to: ./results/dec/DEC_model_960.h5


...

/Users/ra1077ba/miniconda3/envs/tf2_env/lib/python3.8/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


delta_label  0.0 < tol  1e-05
Reached tolerance threshold. Stopping training.
saving model to: ./results/dec/DEC_model_final.h5
Begin pretraining:  ------------------------------------------------------------
Epoch 1/200
25/25 [==============================] - 0s 1ms/step - loss: 0.4721
Epoch 2/200
25/25 [==============================] - 0s 893us/step - loss: 0.2484
Epoch 3/200
25/25 [==============================] - 0s 998us/step - loss: 0.1546
Epoch 4/200
25/25 [==============================] - 0s 978us/step - loss: 0.1043
Epoch 5/200
25/25 [==============================] - 0s 1ms/step - loss: 0.0805
Epoch 6/200
25/25 [==============================] - 0s 1ms/step - loss: 0.0681
Epoch 7/200
25/25 [==============================] - 0s 1ms/step - loss: 0.0628
Epoch 8/200
25/25 [==============================] - 0s 1ms/step - loss: 0.0598
Epoch 9/200
25/25 [==============================] - 0s 1ms/step - loss: 0.0577
Epoch 10/200
25/25 [==============================] - 0s 1ms/step

Pretraining time:  6.541964054107666
Pretrained weights are saved to results/temp/ae_weights.h5
End pretraining:  ------------------------------------------------------------
Begin clustering: ------------------------------------------------------------
Update interval 40
Save interval 10000
Initializing cluster centers with k-means.
97/97 [==============================] - 0s 442us/step
saving model to: ./results/temp/model_0.h5


saving model to: ./results/temp/model_final.h5
Clustering time: 47s
End clustering: ------------------------------------------------------------
...Pretraining...
Epoch 1/40
25/25 [==============================] - 0s 2ms/step - loss: 0.4765
Epoch 2/40
25/25 [==============================] - 0s 1ms/step - loss: 0.2466
Epoch 3/40
25/25 [==============================] - 0s 1ms/step - loss: 0.1524
Epoch 4/40
25/25 [==============================] - 0s 1ms/step - loss: 0.1031
Epoch 5/40
25/25 [==============================] - 0s 1ms/step - loss: 0.0819
Epoch 6/40
25/25 [==============================] - 0s 1ms/step - loss: 0.0702
Epoch 7/40
25/25 [==============================] - 0s 1ms/step - loss: 0.0646
Epoch 8/40
25/25 [==============================] - 0s 1ms/step - loss: 0.0612
Epoch 9/40
25/25 [==============================] - 0s 1ms/step - loss: 0.0590
Epoch 10/40
25/25 [==============================] - 0s 1ms/step - loss: 0.0575
Epoch 11/40
25/25 [===========================

Pretrained weights are saved to ./ae_weights.h5
Update interval 40
Save interval 120
Initializing cluster centers with k-means.
97/97 [==============================] - 0s 249us/step
saving model to: ./results/dec/DEC_model_0.h5
saving model to: ./results/dec/DEC_model_120.h5
saving model to: ./results/dec/DEC_model_240.h5
saving model to: ./results/dec/DEC_model_360.h5
saving model to: ./results/dec/DEC_model_480.h5
saving model to: ./results/dec/DEC_model_600.h5
saving model to: ./results/dec/DEC_model_720.h5
saving model to: ./results/dec/DEC_model_840.h5
saving model to: ./results/dec/DEC_model_960.h5
saving model to: ./results/dec/DEC_model_1080.h5
saving model to: ./results/dec/DEC_model_1200.h5
saving model to: ./results/dec/DEC_model_1320.h5
saving model to: ./results/dec/DEC_model_1440.h5
saving model to: ./results/dec/DEC_model_1560.h5
saving model to: ./results/dec/DEC_model_1680.h5
saving model to: ./results/dec/DEC_model_1800.h5
saving model to: ./results/dec/DEC_model_192

In [201]:
# Assuming your results dictionary is structured as before
for key, y_pred in results.items():
    # Create a copy of the dataset and add the cluster results
    result_df = OH_data_No_aux.copy()
    result_df['cluster'] = y_pred

    # Construct the filename based on the key
    filename = f"../Intermediates/SavedObjects/result_{key}.csv"

    # Save the DataFrame to CSV
    result_df.to_csv(filename, index=None, header=True)

    print(f"Saved results to {filename}")


Saved results to ../Intermediates/SavedObjects/result_FcIDEC_clusters_3.csv
Saved results to ../Intermediates/SavedObjects/result_DEC_clusters_3.csv
Saved results to ../Intermediates/SavedObjects/result_FcIDEC_clusters_4.csv
Saved results to ../Intermediates/SavedObjects/result_DEC_clusters_4.csv
Saved results to ../Intermediates/SavedObjects/result_FcIDEC_clusters_6.csv
Saved results to ../Intermediates/SavedObjects/result_DEC_clusters_6.csv


# OH asthamtics without aux data

In [9]:
# pd.set_option('display.max_columns', 30)
OH_data_No_aux_as = pd.read_csv('../Intermediates/SavedObjects/OH_data_no_aux_as.csv')
del OH_data_No_aux_as['Unnamed: 0']
OH_data_No_aux_as.head()

,gina_steps,education,smoking_status,Age,BMI,onset_age_asthma,FEV1_FVC_ratio_post,Reversibility,tlco,kco,...,smoking_exposure_at_home_Yes,smoking_exposure_at_work_Yes,household_damage_Yes,exertion_triggered_asthma_Yes,infection_triggered_asthma_Yes,allergen_triggered_asthma_Yes,irritant_triggered_asthma_Yes,weather_triggered_asthma_Yes,emotion_triggered_asthma_Yes,XCOPD_Yes
0,1,1,1,30,21.728395,7,0.780767,4.429347,120.376880,105.486292,...,0,1,1,1,1,1,0,0,0,0
1,2,1,1,28,22.595641,3,0.853547,6.240592,121.049749,118.599749,...,1,0,0,1,0,0,0,0,0,0
2,1,1,2,35,27.120316,13,0.864331,12.741017,123.995834,112.723202,...,1,1,0,1,1,1,1,1,0,0
3,1,2,3,21,25.711662,13,0.862000,11.164253,123.992163,98.768946,...,0,1,0,1,0,0,0,0,0,0
4,1,1,1,35,23.566632,14,0.887437,3.559006,130.655238,122.138432,...,0,0,0,0,0,1,0,0,0,0


In [10]:
# OH_data_No_aux.dtypes
# Load the numerical variable names
num_var = pd.read_csv('../Intermediates/SavedObjects/numerical_vars.csv')
# Convert the names to a list
num_var_list = num_var.iloc[:, 0].tolist()

In [11]:
# Assuming num_var_list is already defined and contains the names of numerical columns

# Scale only the numerical columns
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_numerical_data_as = scaler.fit_transform(OH_data_No_aux_as[num_var_list])

# Create a DataFrame from the scaled numerical data
scaled_numerical_df_as = pd.DataFrame(scaled_numerical_data_as, columns=num_var_list)

# Drop the original numerical columns from the main DataFrame
OH_data_No_aux_as = OH_data_No_aux_as.drop(columns=num_var_list)

# Concatenate the scaled numerical DataFrame with the main DataFrame
OH_data_No_aux_scaled_as = pd.concat([OH_data_No_aux_as, scaled_numerical_df_as], axis=1)
OH_data_No_aux_scaled_as

,gina_steps,education,smoking_status,Gender_Yes,Cough_Yes,Breathlessness_Yes,Recurrent_wheeze_Yes,dyspenea_Yes,Chronic_productive_cough_Yes,spt_any_Yes,...,Reversibility,tlco,kco,FeNO,Eosinophiles,Neutrophiles,Lymphocytes,Monocytes,Basophiles,pack_years
0,1,1,1,0,0,0,0,0,0,1,...,-0.378334,1.114154,0.421907,-0.031692,-0.726126,1.767534,0.464392,-0.819459,-0.677048,-0.420836
1,2,1,1,1,0,1,0,0,0,1,...,-0.142343,1.151203,1.271273,-0.389147,1.094714,-1.414777,-0.267630,0.477143,1.483622,-0.845600
2,1,1,2,1,0,1,0,1,0,1,...,0.704610,1.313417,0.890646,-0.576811,-0.726126,-0.619200,-0.999652,-0.171158,-0.677048,-0.279248
3,1,2,3,1,1,1,0,0,1,1,...,0.499170,1.313215,-0.013180,1.205995,0.487767,-1.197802,1.489223,-0.171158,1.483622,-0.647377
4,1,1,1,0,0,1,0,0,0,1,...,-0.491732,1.680090,1.500476,1.107695,-0.726126,-0.763850,-0.267630,-0.171158,-0.677048,-0.675694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1890,1,2,1,1,1,1,1,0,1,0,...,0.454433,1.354026,1.360176,-0.424893,-0.119179,1.767534,-0.706844,0.477143,-0.677048,0.556122
1891,4,1,2,0,1,0,0,0,0,0,...,-1.101019,-1.244317,-1.681632,-0.214888,-0.119179,0.610330,-0.560439,1.773745,-0.677048,3.189660
1892,1,1,1,0,0,0,0,0,0,0,...,-0.540720,1.656105,0.685786,-0.156801,-0.786821,-1.233964,-1.146057,-1.467760,-0.677048,-0.753568
1893,1,1,1,0,0,0,0,0,0,0,...,-0.372960,0.219202,-0.156749,0.111290,-0.847515,-0.980826,-0.853248,-1.467760,-0.677048,-0.350042


In [12]:
# Define your dataset
OH_data_No_aux = OH_data_No_aux_scaled_as  # Load your dataset here

# Define the configurations
n_clusters_list = [4,5]
model_types = ['FcIDEC', 'DEC']
epochs = 1000

# Dictionary to store the results
results = {}

# Loop over each combination of n_clusters and model_type
for n_clusters in n_clusters_list:
    for model_type in model_types:
        # Key for storing results
        key = f"{model_type}_clusters_{n_clusters}"

        # Run the clustering function
        y_pred = cluster_data(data=OH_data_No_aux, n_clusters=n_clusters, neurons=[500, 20], model_type=model_type, epochs=epochs)

        # Store the results
        results[key] = y_pred

# Now, results dictionary contains all the outcomes
# Access the results like results['FcIDEC_clusters_3'], results['DEC_clusters_4'], etc.

/Users/ra1077ba/miniconda3/envs/tf2_env/lib/python3.8/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Begin pretraining:  ------------------------------------------------------------
Epoch 1/200
15/15 [==============================] - 0s 1ms/step - loss: 0.5668
Epoch 2/200
15/15 [==============================] - 0s 1ms/step - loss: 0.3480
Epoch 3/200
15/15 [==============================] - 0s 1ms/step - loss: 0.2481
Epoch 4/200
15/15 [==============================] - 0s 1ms/step - loss: 0.1810
Epoch 5/200
15/15 [==============================] - 0s 973us/step - loss: 0.1414
Epoch 6/200
15/15 [==============================] - 0s 1ms/step - loss: 0.1145
Epoch 7/200
15/15 [==============================] - 0s 1ms/step - loss: 0.0968
Epoch 8/200
15/15 [==============================] - 0s 1ms/step - loss: 0.0848
Epoch 9/200
15/15 [==============================] - 0s 1ms/step - loss: 0.0778
Epoch 10/200
15/15 [==============================] - 0s 1ms/step - loss: 0.0737
Epoch 11/200
15/15 [==============================] - 0s 1ms/step - loss: 0.0713
Epoch 12/200
15/15 [===============

Pretraining time:  4.284278154373169
Pretrained weights are saved to results/temp/ae_weights.h5
End pretraining:  ------------------------------------------------------------
Begin clustering: ------------------------------------------------------------
Update interval 40
Save interval 10000
Initializing cluster centers with k-means.
60/60 [==============================] - 0s 312us/step
saving model to: ./results/temp/model_0.h5


saving model to: ./results/temp/model_final.h5
Clustering time: 41s
End clustering: ------------------------------------------------------------
...Pretraining...
Epoch 1/40
15/15 [==============================] - 0s 1ms/step - loss: 0.5828
Epoch 2/40
15/15 [==============================] - 0s 963us/step - loss: 0.3435
Epoch 3/40
15/15 [==============================] - 0s 919us/step - loss: 0.2549
Epoch 4/40
15/15 [==============================] - 0s 937us/step - loss: 0.1886
Epoch 5/40
15/15 [==============================] - 0s 916us/step - loss: 0.1414
Epoch 6/40
15/15 [==============================] - 0s 916us/step - loss: 0.1151
Epoch 7/40
15/15 [==============================] - 0s 1ms/step - loss: 0.0994
Epoch 8/40
15/15 [==============================] - 0s 1ms/step - loss: 0.0892
Epoch 9/40
15/15 [==============================] - 0s 1ms/step - loss: 0.0821
Epoch 10/40
15/15 [==============================] - 0s 1ms/step - loss: 0.0768
Epoch 11/40
15/15 [=================

Pretrained weights are saved to ./ae_weights.h5
Update interval 40
Save interval 70
Initializing cluster centers with k-means.
60/60 [==============================] - 0s 252us/step
saving model to: ./results/dec/DEC_model_0.h5
saving model to: ./results/dec/DEC_model_70.h5
saving model to: ./results/dec/DEC_model_140.h5
saving model to: ./results/dec/DEC_model_210.h5
saving model to: ./results/dec/DEC_model_280.h5
saving model to: ./results/dec/DEC_model_350.h5
saving model to: ./results/dec/DEC_model_420.h5
saving model to: ./results/dec/DEC_model_490.h5
saving model to: ./results/dec/DEC_model_560.h5
saving model to: ./results/dec/DEC_model_630.h5
saving model to: ./results/dec/DEC_model_700.h5
saving model to: ./results/dec/DEC_model_770.h5
saving model to: ./results/dec/DEC_model_840.h5
saving model to: ./results/dec/DEC_model_910.h5
saving model to: ./results/dec/DEC_model_980.h5
saving model to: ./results/dec/DEC_model_1050.h5
saving model to: ./results/dec/DEC_model_1120.h5


/Users/ra1077ba/miniconda3/envs/tf2_env/lib/python3.8/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


delta_label  0.0 < tol  1e-05
Reached tolerance threshold. Stopping training.
saving model to: ./results/dec/DEC_model_final.h5
Begin pretraining:  ------------------------------------------------------------
Epoch 1/200
15/15 [==============================] - 0s 1ms/step - loss: 0.5668
Epoch 2/200
15/15 [==============================] - 0s 912us/step - loss: 0.3480
Epoch 3/200
15/15 [==============================] - 0s 921us/step - loss: 0.2481
Epoch 4/200
15/15 [==============================] - 0s 898us/step - loss: 0.1810
Epoch 5/200
15/15 [==============================] - 0s 981us/step - loss: 0.1414
Epoch 6/200
15/15 [==============================] - 0s 1ms/step - loss: 0.1145
Epoch 7/200
15/15 [==============================] - 0s 1ms/step - loss: 0.0968
Epoch 8/200
15/15 [==============================] - 0s 1ms/step - loss: 0.0848
Epoch 9/200
15/15 [==============================] - 0s 1ms/step - loss: 0.0778
Epoch 10/200
15/15 [==============================] - 0s 1ms/st

Pretraining time:  4.378131866455078
Pretrained weights are saved to results/temp/ae_weights.h5
End pretraining:  ------------------------------------------------------------
Begin clustering: ------------------------------------------------------------
Update interval 40
Save interval 10000
Initializing cluster centers with k-means.
60/60 [==============================] - 0s 280us/step
saving model to: ./results/temp/model_0.h5


saving model to: ./results/temp/model_final.h5
Clustering time: 40s
End clustering: ------------------------------------------------------------
...Pretraining...
Epoch 1/40
15/15 [==============================] - 0s 1ms/step - loss: 0.5828
Epoch 2/40
15/15 [==============================] - 0s 1ms/step - loss: 0.3435
Epoch 3/40
15/15 [==============================] - 0s 1ms/step - loss: 0.2549
Epoch 4/40
15/15 [==============================] - 0s 1ms/step - loss: 0.1886
Epoch 5/40
15/15 [==============================] - 0s 1ms/step - loss: 0.1414
Epoch 6/40
15/15 [==============================] - 0s 986us/step - loss: 0.1151
Epoch 7/40
15/15 [==============================] - 0s 1ms/step - loss: 0.0994
Epoch 8/40
15/15 [==============================] - 0s 1ms/step - loss: 0.0892
Epoch 9/40
15/15 [==============================] - 0s 1ms/step - loss: 0.0821
Epoch 10/40
15/15 [==============================] - 0s 1ms/step - loss: 0.0768
Epoch 11/40
15/15 [=========================

Pretrained weights are saved to ./ae_weights.h5
Update interval 40
Save interval 70
Initializing cluster centers with k-means.
60/60 [==============================] - 0s 279us/step
saving model to: ./results/dec/DEC_model_0.h5
saving model to: ./results/dec/DEC_model_70.h5
saving model to: ./results/dec/DEC_model_140.h5
saving model to: ./results/dec/DEC_model_210.h5
saving model to: ./results/dec/DEC_model_280.h5
saving model to: ./results/dec/DEC_model_350.h5
saving model to: ./results/dec/DEC_model_420.h5
saving model to: ./results/dec/DEC_model_490.h5
saving model to: ./results/dec/DEC_model_560.h5
saving model to: ./results/dec/DEC_model_630.h5
saving model to: ./results/dec/DEC_model_700.h5
saving model to: ./results/dec/DEC_model_770.h5
saving model to: ./results/dec/DEC_model_840.h5
saving model to: ./results/dec/DEC_model_910.h5
saving model to: ./results/dec/DEC_model_980.h5
saving model to: ./results/dec/DEC_model_1050.h5
saving model to: ./results/dec/DEC_model_1120.h5
sav

In [13]:
# Assuming your results dictionary is structured as before
for key, y_pred in results.items():
    # Create a copy of the dataset and add the cluster results
    result_df = OH_data_No_aux.copy()
    result_df['cluster'] = y_pred

    # Construct the filename based on the key
    # HP for High parameter similar to the all data (both asthmatic and non asthmatic case)
    filename = f"../Intermediates/SavedObjects/result_as_HP{key}.csv"

    # Save the DataFrame to CSV
    result_df.to_csv(filename, index=None, header=True)

    print(f"Saved results to {filename}")


Saved results to ../Intermediates/SavedObjects/result_as_HPFcIDEC_clusters_4.csv
Saved results to ../Intermediates/SavedObjects/result_as_HPDEC_clusters_4.csv
Saved results to ../Intermediates/SavedObjects/result_as_HPFcIDEC_clusters_5.csv
Saved results to ../Intermediates/SavedObjects/result_as_HPDEC_clusters_5.csv


In [22]:
for key, y_pred in results.items():
    # Create a copy of the dataset and add the cluster results
    print(np.unique(y_pred, return_counts= True))


(array([0, 1, 2]), array([505, 565, 825]))
(array([0, 1, 2]), array([787, 487, 621]))


In [25]:
for key, y_pred in results.items():
    # Create a copy of the dataset and add the cluster results
    print(np.unique(y_pred, return_counts= True))


(array([0, 1, 2]), array([955, 397, 543]))
(array([0, 1, 2]), array([559, 656, 680]))
